<a href="https://colab.research.google.com/github/suyash091/covid-chestxray-dataset/blob/master/high_accuracy_best_of_COVID_X_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone 'https://github.com/suyash091/covid-chestxray-dataset'
!git clone https://github.com/qubvel/efficientnet.git

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 817 (delta 2), reused 0 (delta 0), pack-reused 769
Receiving objects: 100% (817/817), 152.42 MiB | 38.20 MiB/s, done.
Resolving deltas: 100% (240/240), done.
Checking out files: 100% (377/377), done.
Cloning into 'efficientnet'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 292 (delta 4), reused 5 (delta 1), pack-reused 280
Receiving objects: 100% (292/292), 835.16 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [0]:
from numpy.random import seed
import random
seed(42)
import os
os.environ['PYTHONHASHSEED'] = str(42)
random.seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

import efficientnet.efficientnet.keras as efn
import pandas as pd

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout, BatchNormalization
import numpy as np
import keras
from keras.optimizers import SGD, adam, Adadelta

In [0]:
def target(string):
  if string=='COVID-19':
    return string
  elif string=='No Finding':
    return 'Nothing'
  else:
    return "Others"

In [0]:
df=pd.read_csv('/content/covid-chestxray-dataset/metadata.csv')
df= df[['filename','finding']]
df['finding']=df['finding'].apply(target)
dfother=pd.read_csv('/content/covid-chestxray-dataset/Others.csv')
dfother.index +=len(df)
dfnothing=pd.read_csv('/content/covid-chestxray-dataset/Nothing.csv')
dfnothing.index +=int(len(df)+len(dfother))
dfbal=pd.read_csv('/content/covid-chestxray-dataset/added.csv')
dfbal.index +=int(len(df)+len(dfother)+len(dfnothing))

In [0]:
frames = [df, dfother, dfnothing, dfbal]

df = pd.concat(frames)
df= df[['filename','finding']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
df.describe()

,filename,finding
count,365,365
unique,364,3
top,person1003_virus_1685.jpeg,Others
freq,2,123


In [0]:
len(df)

365

In [0]:
df.finding.unique()

array(['COVID-19', 'Others', 'Nothing'], dtype=object)

In [0]:
len(df[df['finding']=='COVID-19'])-len(df[df['finding']=='Others'])

-2

In [0]:
from keras_preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split=0.1)

#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory='/content/covid-chestxray-dataset/images',
        x_col="filename",
        y_col='finding',
        target_size=(1024, 1024),
        batch_size=16,
        subset = 'training' ,
        class_mode='categorical')

validation_generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory='/content/covid-chestxray-dataset/images',
        x_col="filename",
        y_col='finding',
        target_size=(1024, 1024),
        batch_size=16,
        subset="validation",
        class_mode='categorical')

Found 328 validated image filenames belonging to 3 classes.
Found 36 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [0]:
efficient_net = efn.EfficientNetB7(
    weights='imagenet',
    input_shape=(1024,1024,3),
    include_top=False
)
efficient_net.trainable = False













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
258441216/258434480 [==============================] - 7s 0us/step


In [0]:
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten

In [0]:
import tensorflow as tf
B = tf.keras.backend

@tf.function(experimental_relax_shapes=True)
def gelu(x):
    return 0.5 * x * (1 + B.tanh(x * 0.7978845608 * (1 + 0.044715 * x * x)))

@tf.function
def fast_gelu(x):
    return B.hard_sigmoid(1.702 * x) * x

In [0]:
from keras import backend as K

def swish(x, beta=1.0):
    return x * K.sigmoid(beta * x)

In [0]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

AveragePooling + Flatten





In [0]:
##CNN + FC 256 x 1 + AP + FLatten + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(AveragePooling2D(pool_size=(4, 4)))
model.add(Flatten(name="flatten"))
#model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 256))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.5))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m]) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.9594 val == 0.8000

Epoch 1/5000
20/20 [==============================] - 90s 4s/step - loss: 0.9646 - acc: 0.6156 - f1_m: 0.5759 - precision_m: 0.6244 - recall_m: 0.5411 - val_loss: 0.7546 - val_acc: 0.6250 - val_f1_m: 0.6351 - val_precision_m: 0.6458 - val_recall_m: 0.6250
Epoch 2/5000
20/20 [==============================] - 52s 3s/step - loss: 0.7140 - acc: 0.7439 - f1_m: 0.7456 - precision_m: 0.7760 - recall_m: 0.7190 - val_loss: 1.0171 - val_acc: 0.5000 - val_f1_m: 0.4184 - val_precision_m: 0.4400 - val_recall_m: 0.4000
Epoch 3/5000
20/20 [==============================] - 51s 3s/step - loss: 0.6591 - acc: 0.7749 - f1_m: 0.7679 - precision_m: 0.7963 - recall_m: 0.7436 - val_loss: 0.6758 - val_acc: 0.5000 - val_f1_m: 0.5371 - val_precision_m: 0.5810 - val_recall_m: 0.5000
Epoch 4/5000
20/20 [==============================] - 54s 3s/step - loss: 0.4772 - acc: 0.8406 - f1_m: 0.8365 - precision_m: 0.8638 - recall_m: 0.8125 - val_loss: 0.8029 - val_acc: 0.5938 - val_f1_m: 0.5937 - val_precision_m: 0.5938

Process ForkPoolWorker-68:
Process ForkPoolWorker-67:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.

KeyboardInterrupt: ignored

In [0]:
##CNN + FC II

In [0]:
##CNN + PCA + SVM

In [0]:
##CNN + PCA + KNN